In [91]:
import praw
import pandas as pd
import datetime as dt

import sqlite3
import sqlalchemy as db
from sqlalchemy import create_engine

import cProfile

### Defining major functions for scrapping Reddit

In [104]:
def get_reddit(client_id,client_secret,user_agent):
    """Setting up the reddit authentication information"""
    reddit = praw.Reddit(client_id=client_id,client_secret = client_secret,user_agent=user_agent)
    return reddit

In [106]:
def get_posts_result(reddit,keyword,len_of_result):
    """Getting the scrap result and put it into a pandas dataframe"""
    posts = []
    ml_subreddit = reddit.subreddit(keyword)
    for post in ml_subreddit.new():
        if not (post.is_self & len(posts)==len_of_result):
            posts.append([post.id, post.subreddit, post.title, post.selftext, post.url, dt.datetime.now(),post.author,post.created_utc,post.comments])
    posts_df=pd.DataFrame(posts,columns=['post_id','subreddit_name','post_title','post_body','visual_content_url','scrap_time','post_author','post_publish_date','post_comments'])
    return posts_df

In [109]:
def transform_posts_results(posts_df):
    """Prepare the dataframe for Sqlite3"""
    posts_df['post_publish_date'] = pd.to_datetime(posts_df['post_publish_date'],unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    col_name= ["post_id","subreddit_name",'post_title','post_body','visual_content_url','post_author','post_comments']
    posts_df[col_name] = posts_df[col_name].astype(str)
    return posts_df

In [119]:
def main():
    client_id="rKCfOb1SzwpPQQ"
    client_secret = 'Ktrwjri1ugder7wWFRuDwPjitll5Zg'
    user_agent='FixFake_Scraper'
    reddit = get_reddit(client_id,client_secret,user_agent)
    posts_df = get_posts_result(reddit,'Shoes',100)
    posts_df = transform_posts_results(posts_df)
    engine = create_engine('sqlite:///reddit_shoes.db',echo=True)
    sqlite_connection = engine.connect()
    sqlite_table="reddit_shoes"
    posts_df.to_sql(sqlite_table,sqlite_connection,if_exists='replace')
    print("Scrap table exported successfully")
    

### Below is the profile results

In [121]:
cProfile.run("main()")

2020-12-20 16:22:29,102 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-20 16:22:29,103 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 16:22:29,104 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-20 16:22:29,106 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 16:22:29,112 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reddit_shoes")
2020-12-20 16:22:29,114 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 16:22:29,116 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reddit_shoes")
2020-12-20 16:22:29,118 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 16:22:29,120 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-20 16:22:29,122 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 16:22:29,126 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reddit_shoes")
2020-12-20 16:22:29,128 INFO sqlalchemy.engine.bas

2020-12-20 16:22:31,955 INFO sqlalchemy.engine.base.Engine COMMIT
Scrap table exported successfully
         647938 function calls (609215 primitive calls) in 26.071 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      367    0.001    0.000    0.002    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:103(release)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:147(__enter__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:151(__exit__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
       40    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:222(_v

        2    0.000    0.000    0.000    0.000 _util.py:111(byte_string)
      105    0.000    0.000    0.000    0.000 _util.py:127(text_to_bytes_and_warn)
       20    0.000    0.000    0.000    0.000 _util.py:62(openssl_assert)
       96    0.000    0.000    0.000    0.000 _util.py:72(native)
        2    0.000    0.000    0.000    0.000 _util.py:93(path_string)
        1    0.000    0.000    0.000    0.000 _validators.py:231(validate_bool_kwarg)
     2602    0.002    0.000    0.005    0.000 abc.py:137(__instancecheck__)
       16    0.000    0.000    0.000    0.000 abc.py:141(__subclasscheck__)
        2    0.000    0.000    0.000    0.000 accessor.py:171(__get__)
        1    0.000    0.000    0.000    0.000 accessor.py:78(_getter)
        3    0.000    0.000    0.002    0.001 accessor.py:92(f)
        1    0.000    0.000    0.002    0.002 accessors.py:103(_delegate_method)
        2    0.000    0.000    0.000    0.000 accessors.py:27(__init__)
        2    0.000    0.000    0.000  

        4    0.000    0.000    0.000    0.000 base.py:958(__getattr__)
        2    0.000    0.000    0.000    0.000 base.py:969(visit_cast)
        5    0.000    0.000    0.000    0.000 base.py:98(__iter__)
        1    0.000    0.000    0.000    0.000 base64.py:51(b64encode)
       74    0.000    0.000    0.000    0.000 binding.py:55(_openssl_assert)
       31    0.000    0.000    0.000    0.000 blocks.py:120(__init__)
        1    0.000    0.000    0.000    0.000 blocks.py:1294(take_nd)
       31    0.000    0.000    0.000    0.000 blocks.py:131(_check_ndim)
        6    0.000    0.000    0.000    0.000 blocks.py:170(_consolidate_key)
        1    0.000    0.000    0.000    0.000 blocks.py:183(is_datelike)
        1    0.000    0.000    0.000    0.000 blocks.py:188(is_categorical_astype)
        8    0.000    0.000    0.000    0.000 blocks.py:203(external_values)
       23    0.000    0.000    0.000    0.000 blocks.py:207(internal_values)
        1    0.000    0.000    0.000    0.00

        8    0.000    0.000    0.000    0.000 common.py:741(condition)
        2    0.000    0.000    0.000    0.000 common.py:814(is_datetimelike)
       10    0.000    0.000    0.000    0.000 common.py:862(is_dtype_equal)
       10    0.000    0.000    0.000    0.000 common.py:951(is_integer_dtype)
       31    0.000    0.000    0.000    0.000 common.py:99(is_bool_indexer)
       10    0.000    0.000    0.001    0.000 compiler.py:1445(visit_bindparam)
       10    0.000    0.000    0.000    0.000 compiler.py:1529(_truncate_bindparam)
        2    0.000    0.000    0.000    0.000 compiler.py:1542(_truncated_identifier)
        2    0.000    0.000    0.000    0.000 compiler.py:1564(_process_anon)
       10    0.000    0.000    0.000    0.000 compiler.py:1570(bindparam_string)
        2    0.000    0.000    0.000    0.000 compiler.py:1758(_add_to_result_map)
        2    0.000    0.000    0.000    0.000 compiler.py:1761(_label_select_column)
        2    0.000    0.000    0.001    0.000

...

      101    0.001    0.000    1.933    0.019 generator.py:56(__next__)
        1    0.000    0.000    1.932    1.932 generator.py:68(_next_batch)
       26    0.000    0.000    0.000    0.000 generic.py:162(__init__)
        3    0.000    0.000    0.000    0.000 generic.py:185(_init_mgr)
        1    0.000    0.000    0.000    0.000 generic.py:1934(__contains__)
        1    0.000    0.000    2.985    2.985 generic.py:2562(to_sql)
        8    0.000    0.000    0.001    0.000 generic.py:3265(_get_item_cache)
       17    0.000    0.000    0.000    0.000 generic.py:3279(_set_as_cached)
       10    0.000    0.000    0.000    0.000 generic.py:3358(_clear_item_cache)
        8    0.000    0.000    0.001    0.000 generic.py:3380(_set_item)
        1    0.000    0.000    0.000    0.000 generic.py:3384(_set_is_copy)
        8    0.000    0.000    0.000    0.000 generic.py:3413(_check_setitem_copy)
        1    0.000    0.000    0.001    0.001 generic.py:3524(take)
        2    0.000    0.00

        2    0.000    0.000    0.000    0.000 numeric.py:228(inferred_type)
        1    0.000    0.000    0.000    0.000 numeric.py:238(_convert_scalar_indexer)
    51/49    0.000    0.000    0.000    0.000 numeric.py:469(asarray)
       13    0.000    0.000    0.000    0.000 numeric.py:541(asanyarray)
        6    0.000    0.000    0.000    0.000 numeric.py:676(require)
        6    0.000    0.000    0.000    0.000 numeric.py:748(<setcomp>)
      136    0.000    0.000    0.000    0.000 numerictypes.py:293(issubclass_)
       68    0.000    0.000    0.000    0.000 numerictypes.py:365(issubdtype)
        2    0.000    0.000    0.000    0.000 numerictypes.py:578(_can_coerce_all)
        8    0.000    0.000    0.000    0.000 numerictypes.py:587(<listcomp>)
        1    0.000    0.000    0.000    0.000 numerictypes.py:602(find_common_type)
        1    0.000    0.000    0.000    0.000 numerictypes.py:654(<listcomp>)
        1    0.000    0.000    0.000    0.000 numerictypes.py:655(<listco

        3    0.000    0.000    0.000    0.000 selectable.py:601(columns)
        1    0.000    0.000    0.000    0.000 series.py:1065(__getitem__)
        1    0.000    0.000    0.000    0.000 series.py:1112(_get_with)
        1    0.000    0.000    0.000    0.000 series.py:1177(_get_values)
    22/21    0.000    0.000    0.001    0.000 series.py:194(__init__)
        1    0.000    0.000    0.001    0.001 series.py:320(_init_dict)
        2    0.000    0.000    0.000    0.000 series.py:399(_constructor)
       22    0.000    0.000    0.000    0.000 series.py:416(_set_axis)
       22    0.000    0.000    0.000    0.000 series.py:443(_set_subtyp)
       35    0.000    0.000    0.000    0.000 series.py:453(name)
       23    0.000    0.000    0.000    0.000 series.py:460(name)
       29    0.000    0.000    0.000    0.000 series.py:467(dtype)
        9    0.000    0.000    0.000    0.000 series.py:517(values)
       23    0.000    0.000    0.000    0.000 series.py:559(_values)
        1  

      102    0.000    0.000    0.002    0.000 utils.py:589(requote_uri)
      204    0.000    0.000    0.000    0.000 utils.py:672(set_environ)
      102    0.002    0.000    0.115    0.001 utils.py:694(should_bypass_proxies)
      102    0.000    0.000    0.002    0.000 utils.py:702(<lambda>)
      102    0.000    0.000    0.201    0.002 utils.py:755(get_environ_proxies)
      204    0.002    0.000    0.010    0.000 utils.py:767(select_proxy)
        1    0.000    0.000    0.000    0.000 utils.py:793(default_user_agent)
        1    0.000    0.000    0.000    0.000 utils.py:802(default_headers)
      101    0.001    0.000    0.003    0.000 utils.py:906(get_auth_from_url)
      509    0.001    0.000    0.002    0.000 utils.py:927(check_header_validity)
      3/2    0.000    0.000    2.356    1.178 visitors.py:127(traverse_single)
        6    0.000    0.000    0.000    0.000 visitors.py:154(visitor_iterator)
        3    0.000    0.000    0.000    0.000 visitors.py:222(iterate)
       

        1    0.000    0.000    0.000    0.000 {method 'keys' of 'collections.OrderedDict' objects}
     1221    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        8    0.000    0.000    0.000    0.000 {method 'lower' of 'bytes' objects}
    46282    0.015    0.000    0.015    0.000 {method 'lower' of 'str' objects}
     2242    0.001    0.000    0.001    0.000 {method 'lstrip' of 'str' objects}
     3200    0.007    0.000    0.007    0.000 {method 'match' of 're.Pattern' objects}
       36    0.000    0.000    0.000    0.000 {method 'new' of 'CompiledFFI' objects}
        4    0.000    0.000    0.000    0.000 {method 'newbyteorder' of 'numpy.dtype' objects}
        1    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     4514    0.002    0.000    0.002    0.000 {method 'partition' of 'str' objects}
      214   21.816    0.102   21.816    0.102 {method 'poll' of 'select.poll' objects}
      102    0.000    0.000    0.000    0.000 {

In [115]:
# View the database info 
table_df = pd.read_sql_table("reddit_shoes",con=engine)
print(table_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
index                 100 non-null int64
post_id               100 non-null object
subreddit_name        100 non-null object
post_title            100 non-null object
post_body             100 non-null object
visual_content_url    100 non-null object
scrap_time            100 non-null datetime64[ns]
post_author           100 non-null object
post_publish_date     100 non-null object
post_comments         100 non-null object
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 7.9+ KB
None


In [116]:
!pip freeze > requirements.txt